In [1]:
?@generated

```
@generated f
@generated(f)
```

`@generated` is used to annotate a function which will be generated. In the body of the generated function, only types of arguments can be read (not the values). The function returns a quoted expression evaluated when the function is called. The `@generated` macro should not be used on functions mutating the global scope or depending on mutable elements.

See [Metaprogramming](@ref) for further details.

## Example:

```julia
julia> @generated function bar(x)
           if x <: Integer
               return :(x ^ 2)
           else
               return :(x)
           end
       end
bar (generic function with 1 method)

julia> bar(4)
16

julia> bar("baz")
"baz"
```


In [2]:
function _staticsum(f, a, b)
    expr = :(f($a))
    for i in a+1:b
        expr = :($expr + f($i))
    end
    expr
end

_staticsum (generic function with 1 method)

In [3]:
_staticsum(:f, -3, 4)

:(((((((f(-3) + f(-2)) + f(-1)) + f(0)) + f(1)) + f(2)) + f(3)) + f(4))

In [4]:
@generated function staticsum(f, ::Val{a}, ::Val{b}) where {a, b}
    @assert b - a < 100
    expr = :(f($a))
    for i in a+1:b
        expr = :($expr + f($i))
    end
    expr
end

staticsum (generic function with 1 method)

In [5]:
staticsum(identity, Val(1), Val(10))

55

In [6]:
2staticsum(x -> sin(x)/x, Val(1), Val(100)) + 1

3.120857876802125

In [7]:
F(n) = 2sum(x -> sin(x)/x, 1:n) + 1
F_static(n) = 2staticsum(x -> sin(x)/x, Val(1), Val(n)) + 1

F_static (generic function with 1 method)

In [8]:
using BenchmarkTools
@show F(100) == F_static(100)
@btime F(100)
@btime F_static(100)

F(100) == F_static(100) = true
  991.667 ns (0 allocations: 0 bytes)
  2.000 ns (0 allocations: 0 bytes)


3.120857876802125

In [9]:
@benchmark F(100)

BenchmarkTools.Trial: 10000 samples with 12 evaluations.
 Range (min … max):  983.333 ns …   5.067 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):       1.025 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.054 μs ± 227.020 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                              
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▁▁▁▁▁ ▁ ▁
  1.35 μs          Histogram: frequency by time         1.02 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [10]:
@benchmark F_static(100)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.900 ns … 39.300 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.000 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.132 ns ±  1.112 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                           
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  2.9 ns         Histogram: frequency by time           2 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [11]:
@code_warntype staticsum(x -> sin(x)/x, Val(1), Val(5))

Variables
  #self#::Core.Const(staticsum)
  f::Core.Const(var"#8#9"())
  #unused#@_3::Core.Const(Val{1}())
  #unused#@_4::Core.Const(Val{5}())

Body::Float64
1 ─ %1 = (f)(1)::Float64
│   %2 = (f)(2)::Float64
│   %3 = (%1 + %2)::Float64
│   %4 = (f)(3)::Float64
│   %5 = (%3 + %4)::Float64
│   %6 = (f)(4)::Float64
│   %7 = (%5 + %6)::Float64
│   %8 = (f)(5)::Float64
│   %9 = (%7 + %8)::Float64
└──      return %9


In [12]:
@code_typed staticsum(x -> sin(x)/x, Val(1), Val(5))

CodeInfo(
1 ─ %1  = Base.muladd_float(0.3258084466825465, 2.480158728947673e-5, -0.001388888888887411)::Float64
│   %2  = Base.muladd_float(0.3258084466825465, %1, 0.0416666666666666)::Float64
│   %3  = Base.mul_float(0.3258084466825465, %2)::Float64
│   %4  = Base.muladd_float(0.3258084466825465, -1.1359647557788195e-11, 2.087572321298175e-9)::Float64
│   %5  = Base.muladd_float(0.3258084466825465, %4, -2.7557314351390663e-7)::Float64
│   %6  = Base.mul_float(0.10615114392969374, 0.10615114392969374)::Float64
│   %7  = Base.mul_float(%6, %5)::Float64
│   %8  = Base.add_float(%3, %7)::Float64
│   %9  = Base.mul_float(0.3258084466825465, %8)::Float64
│   %10 = Base.sub_float(%9, -2.8419927711204204e-17)::Float64
│   %11 = Base.add_float(-5.551115123125783e-17, %10)::Float64
│   %12 = Base.add_float(0.8370957766587268, %11)::Float64
│   %13 = Base.div_float(%12, 1.0)::Float64
│   %14 = Base.muladd_float(0.18421579309275318, 2.480158728947673e-5, -0.001388888888887411)::Float64
│   %15 = 

In [13]:
@code_typed sum(x -> sin(x)/x, 1:5)

CodeInfo(
1 ─ %1 = Base.add_sum::typeof(Base.add_sum)
│   %2 = invoke Base._mapreduce(_2::var"#12#13", %1::typeof(Base.add_sum), $(QuoteNode(IndexLinear()))::IndexLinear, _3::UnitRange{Int64})::Float64
└──      return %2
) => Float64